In [ ]:
import earthaccess

import os
try:
    os.chdir(base_dir) #if the start up file has already been run, this will change the dir back to base
except NameError:
    base_dir = None #if the start up file hasn't been run yet, this will allow us to run this cell without crashing

%run ./startup_file.ipynb #run startup file and install libraries if necessary

In [ ]:
auth = earthaccess.login(persist=True)

In [ ]:
results = earthaccess.search_datasets(instrument="oci")

In [ ]:
for item in results:
    summary = item.summary()
    print(summary["short-name"])

In [ ]:
results = earthaccess.search_data(
    short_name="PACE_OCI_L2_AOP_NRT",
    count=1,
)

In [ ]:
tspan = ("2024-04-01", "2024-04-16")
bbox = (-77.7, 34.4, -63.2, 46.5)
clouds = (0, 50)

results = earthaccess.search_data(
    short_name="PACE_OCI_L2_AOP_NRT",
    temporal=tspan,
    bounding_box=bbox, 
    cloud_cover=clouds,
)

In [ ]:
paths = earthaccess.open(results)

In [ ]:
with paths[0] as file:
    line = file.readline().strip()
line

In [ ]:
paths = earthaccess.download(results, "L2_AOP")

In [15]:
paths

['L2_AOP\\PACE_OCI.20240413T175156.L2.OC_AOP.V1_0_0.NRT.nc',
 'L2_AOP\\PACE_OCI.20240413T175656.L2.OC_AOP.V1_0_0.NRT.nc',
 'L2_AOP\\PACE_OCI.20240414T182658.L2.OC_AOP.V1_0_0.NRT.nc',
 'L2_AOP\\PACE_OCI.20240414T183158.L2.OC_AOP.V1_0_0.NRT.nc']

In [20]:
dataset = xr.open_dataset(paths[0], group="geophysical_data")
rrs = dataset["Rrs"]
dataset = xr.open_dataset(paths[0], group="navigation_data")
dataset = dataset.set_coords(("longitude", "latitude"))
dataset = dataset.rename({"pixel_control_points": "pixels_per_line"})
#dataset = xr.merge((rrs, dataset.coords))
#dataset
rrs

<xarray.DataArray 'Rrs' (number_of_lines: 1709, pixels_per_line: 1272,
                         wavelength_3d: 184)>
[399988032 values with dtype=float32]
Dimensions without coordinates: number_of_lines, pixels_per_line, wavelength_3d
Attributes:
    long_name:      Remote sensing reflectance
    units:          sr^-1
    standard_name:  surface_ratio_of_upwelling_radiance_emerging_from_sea_wat...
    valid_min:      -30000
    valid_max:      25000

In [ ]:
from scipy.interpolate import griddata
grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
